| | |
|---|---|
| **Name** | Jan Ubbo van Baardewijk & Thomas Brus |
| **Group** | ML_HMI_01 |


$
\newcommand{\x}{\mathbf{x}}
\newcommand{\w}{\mathbf{w}}
\newcommand{\z}{\mathbf{z}}
\newcommand{\wx}{\w^\top\x}
\newcommand{\dataset}{\mathbf{X}}
\newcommand{\tset}{t\negthickspace t}
$

# Week 4: Neural networks

The basic unit of a multi-layer perceptron (MLP) is the neuron, which is a computational unit that computes a weighted sum of its inputs $\wx$ and pushes this through a non-linear activation function. Important is that this activation should be differentiable, as we will train the weights of the network by gradient descent. This requires that we should be able to compute the gradient of an error function with respect to the weights. 

In a sense, this whole lab is about taking derivatives and applying the chain rule of derivation. As a reminder, the rule is that the derivative of a function $f$ applied to the result of a function $g$ $f(g(x))$ can be decomposed as:
$$
\frac{\partial}{\partial x} f(g(x)) = \frac{\partial f(g(x))}{\partial g(x)} \, \frac{\partial g(x)}{\partial x} 
$$


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
import copy

maxit=100
def gradDesc(w, errorfn, gradfn,  eta=1e-3, verbose=True):
    err = errorfn(w)             # Compute the error function
    errs = np.zeros(maxit)       # Keep track of the error function
    ts = np.zeros(maxit)         # Keep track of time stamps
    start = time.time()

    newError = errorfn(w) + 1.
    for n in xrange(maxit):
        grad = gradfn(w)
        if verbose:
            print "##", n, "err:", newError, "eta:", eta, "w:", w
            print "  Gradient:", grad
            print "  FD grad: ", gradfd(w,errorfn)
        pw = copy.copy(w)
        w -= grad * eta     # Update the weights
        
        pastError = newError
        newError = errorfn(w)
        while eta > 0. and pastError - newError < 0: # If the error increases, eta is too large
            eta /= 2.0                                # halve it, and
            w = pw - eta * grad                      # try again from the original value of the weights
            newError = errorfn(w)
        else:                                        # If the error goes down,
            eta *= 1.2                               # try to increase eta a little, to speed up things.
                
        if pastError-newError < 1e-5:                # If we couldn't decrease the error anymore, 
            return w, errs[:n], ts[:n]               # just give up
 
        errs[n] = newError                           # Keep track of how the errors evolved
        ts[n] = time.time()-start

    return w, errs, ts


## Neuron definition

In this case, let's set up our network to have the sigmoidal activation function that we've seen before in logistic regression, 
$$
\sigma(a) = \frac{1}{1+e^{-a}}\ ,
$$ 
so that the activation $z_i$ of neuron $i$ is given by
$$
z_i = \sigma(\w_i^\top\x_i) \ ,
$$
where $\x_i$ is the input to neuron $i$. Other activation functions are possibble, such as a linear function ($z=\wx$) or a hyperbolic tangend ($z=\tanh(\wx)$). Let's consider a network of a single neuron, then the output of our network $y(\w,\x) = \sigma(\wx)$

## Training

To train a single neuron, we have a set of datapoints $\dataset = \{ \x_1\dots \x_N\}$ and corresponding targets $\tset = \{ t_1 \dots t_N \}$. We use this dataset to find optimal weights, and we define optimality on terms of an error function. Let's define our error function as a sum of squared errors:
$$
E(\w) = \tfrac{1}{2} \sum_{n=1}^N (y(\w,\x_n) - t_n)^2
$$

**Question [5 credits]**: Notice how, although our single neuron is very similar to the logistic regression we saw in previous weeks, our error function is now quite different. What is now the error for our set $\dataset, \tset$? Write it out algebraically


*Answer to Q1*


**Question [5 credits]** What is the gradient of this error with respects to the weights of the neuron $\w$? Be careful, this is not the same as what we saw in the lecture, where the output neuron had a linear activation function.

*Answer to Q2*


**Question [10 credits]** If now we had a neural network with two neurons in a hidden layer, 
* what would be the error function, analytically? 
* What is the derivative of that error with respect to the weights of the first and second layer?

How do your equations correspond to the quantities computed in backpropagation?

*Answer to Q3*


**Question [10 credits]** Implement the following python functions: 

* `error(outputs, targets)`, which implements the sum-squared error on our dataset, and a function 
* `gradError(outputs,targets)`, which computes the gradient of that error with respect to its input variable `outputs`
* `neuron(weights, inputs)`, wich implements a neuron with sigmoidal activation function, weights $\w$ and inputs $\x$. Decide how you deal with the bias (separately, or by augmenting the input data)
* `gradNeuron(weights, inputs)`, which implements the gradient of the neuron's output with respect to the neuron's weights.

In [3]:
# Answer to Q4

    

** Question [10 credits]** Verify using finite differences that the gradients you implemented for the above functions is actually correct.

In [8]:
## Answer Q5

### Training a single neuron

We now have the code to implement gradient descent on a single neuron. We simply need to apply the chain rule of derivation to get the complete gradient.
** Question [10 marks] ** 
Implement the error function and the gradient of the error function with respect to the weights. Then use last week's code to perform gradient descent on the data provided. Plot the evolution of the error in one plot, and the function that the neuron implements overlayed with the training data (in another plot)


In [4]:
data = np.load("curve.npz")
train = np.array([data['train'][:,0]]).T
traint = data['train'][:,1]

In [5]:
# Answer


## Defining and training a network

We define a feed-forward neural network using the following information:
1. The number of layers in the network
1. The number of nodes in each layer
1. The activation function used in each layer (notice that all nodes in a layer use the same activation function)

The important thing to notice in such a network, is that the input of the nodes in layer $n$ are the outputs of the nodes in layer $n-1$, so that we have a chaining of functions and the chain rule of derivatives applies directly. We can, therefore, apply the chain rule mechanically and compute the gradient using a combination of the forward-propagated inputs and backward-propagated errors.

The code below implements a neural network with arbitrary numbers of hidden nodes, arbitrary numbers of hidden layers, and arbitrary activation functions. Do look at the code and make sure you understand how it works, and how to use it. To use it, activation functions need to be defined. The following cell provides you with two examples. 

In [6]:
# This code is given
# The following functions are activation functions. For efficiency, they combine the computation of 
# the activation function itself and its gradient wrt its input values, as both often rely on computing 
# the same intermediate quantities

def ssegf(y,t):
    '''Error and gradient function for sum-of-squares'''
    return .5*(y-t)**2, (y-t)

def linegf(a):
    '''Error and gradient function for linear activation'''
    return a,np.ones(a.shape)

**Question [10 credits]**: Implement the logistic (sigmoidal) and the hyperbolic tangent "Error and Gradient" activation functions.

In [12]:
# Answer


In [11]:
class NN:
    '''Implementation of an artificial neural network'''
    def __init__(self, nIn, nHidden, nOut, actFn):
        '''Initialise the network with random weights and biases. Parameters are:
        nIn: Number of input nodes (dimensionality of the data)
        nHidden: LIST of numbers of hidden nodes, one integer per layer. Empty list for no hidden layers
        nOut: Number of output nodes; dimensionality of the targets
        actFn: Tuple of activation functions; one function per computational layer (hidden or output)'''
        
        assert(len(nHidden) == len(actFn)-1)    # Check that the parameters are consistent
        
        self.numNodes = [nIn]                   # Create a list of numbers of nodes per layer
        if nHidden:
            self.numNodes.extend(nHidden)
        self.numNodes.append(nOut)

        # Initialise weights and biases
        self.w = [ 1e-0 * np.random.randn(self.numNodes[i],self.numNodes[i-1]) for i in range(1,len(self.numNodes)) ]
        # print "Weights", self.w
        self.biases = [ 1e-0 * np.random.randn(self.numNodes[i]) for i in range(1,len(self.numNodes))]
        # print "Biases:", self.biases
        self.actfn = actFn
        
    def forward(self,x):
        ''' Compute the activations of the nodes of each layer.
            For efficiency, use "EGF" functions and store the gradient of the activation function as well. In
            other words each entry of self.fw is a tuple containing (z_i, h'(a_i))
        '''
        self.fw = [ self.actfn[0](self.w[0].dot(x) + self.biases[0]) ]  # First layer takes data as input
        for h,w,b in zip(self.actfn[1:],self.w[1:],self.biases[1:]):    # following layers take the output of  
            self.fw.append(h(w.dot(self.fw[-1][0]) + b))                #    the previous layer
        return self.fw[-1][0]
    
    def back(self, errorgrad):
        '''Implement backpropagation of the errors'''
        self.delta = [ self.fw[-1][1] * errorgrad ]             # Start with the gradient of the error function
        
        for layer in range(len(self.w)-1,0,-1):
            hprime = self.fw[layer-1][1]                        # h'(a_j)
            delta = self.delta[0]                               # list gets extended at start, first elt: delta_k
            w = self.w[layer]                                   # w_kj

            self.delta.insert(0,hprime * (self.delta[0].dot(w))) # Use dot product to sum over all FOLLOWING weights        

    def gradients(self, x, t, eta=1e-3):
        '''Compute the gradients of the error of a single datapoint, with respect to
        all weights and all biases'''
        y = self.forward(x)                        # Forward propagation and computation of h'
        error = y-t
        self.back(error)                           # Backward propagation
                              
        gradw = []
        gradb = []
        gradw.append(np.outer(self.delta[0],x))    # derivative wrt w_ji = delta_j z_i, where z_i == x_i for input nodes
        gradb.append(self.delta[0])                #    and z_i == 1 for bias nodes
        
        for i in range(1,len(self.w)):
            gradw.append(np.outer(self.delta[i],self.fw[i-1][0])) # for folloing layers, use output of previous layer
            gradb.append(self.delta[i])
        return gradw,gradb,.5*error*error          # Return gradient wrt w and bias, and the error

    def batchGradients(self, data,targets):
        '''Compute the gradients of all weights and biases, for the complete dataset'''                      
        gradw,gradb,error = self.gradients(data[0,:],targets[0])
        for x,t in zip(data[1:,:],targets[1:]):
            tw,tb,e = self.gradients(x,t)
            error += e
            for i in range(len(gradw)):
                gradw[i] += tw[i]
                gradb[i] += tb[i]
        return gradw,gradb,error
    
    def batchStep(self,data,targets,eta=1e-3):
        '''Make one step of batch gradient descent'''                      
        gw,gb,e = self.batchGradients(data,targets)
        for i in range(len(gw)):
            self.w[i] -= eta*gw[i]
            self.biases[i] -= eta*gb[i]
        return e
       
    def train(self,data,targets,eta=1e-4):
        '''Train the network using batch GD'''
        es = np.zeros(100000)
        for i in range(len(es)):
            es[i] = self.batchStep(data,targets,eta)
            if i>100 and es[i-1]< es[i]+1e-5:
                return es[:i+1]
        return es

**Question [10 credits]** Using the above code, create a neural network with no hidden layer and linear output activation function, and train it on the provided data. Plot the evolution of the training error, as well as the function that the neural network implements (in two separate plots). 

In [45]:
# Answer

** Question [30 credits] ** Use the provided test data to compare this to other configurations of your choice (more hidden layers, different activation functions, ...). Find a network that performs well on the given data. Perform multiple training runs with random initialisations, plot the final validation error for each run in function of the number of nodes, and keep the best. Plot the train and test data, as well as the function implemented by your final network

In [ ]:
# Answer